As usual we're start with cleaning and attempt to delay EDA until the end.

In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
df = pd.read_csv('../data/train.csv')
print(df.shape)
df.head()

#1.2 million people... no wonder this was such a big file. Realistic considering the industy and what I've worked with
#in the past.

#Recall that premium is this competion's target.

(1200000, 21)


,id,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,...,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,0,19.00,Female,"10,049.00",Married,1.00,Bachelor's,Self-Employed,22.60,Urban,...,2.00,17.00,372.00,5.00,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,"2,869.00"
1,1,39.00,Female,"31,678.00",Divorced,3.00,Master's,NaN,15.57,Rural,...,1.00,12.00,694.00,2.00,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,"1,483.00"
2,2,23.00,Male,"25,602.00",Divorced,3.00,High School,Self-Employed,47.18,Suburban,...,1.00,14.00,NaN,3.00,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.00
3,3,21.00,Male,"141,855.00",Married,2.00,Bachelor's,NaN,10.94,Rural,...,1.00,0.00,367.00,1.00,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.00
4,4,21.00,Male,"39,651.00",Single,1.00,Bachelor's,Self-Employed,20.38,Rural,...,0.00,8.00,598.00,4.00,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,"2,022.00"


In [28]:
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')

In [29]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Index(['age', 'annual_income', 'marital_status', 'number_of_dependents',
       'occupation', 'health_score', 'previous_claims', 'vehicle_age',
       'credit_score', 'insurance_duration', 'customer_feedback'],
      dtype='object')


In [30]:
#After back and fourth with ChatGPT and I, I figured this approach is better. No need to loop.

thresh = .02

null_counts = pd.DataFrame({'null_sum': df.isnull().sum()})
null_counts['ratio'] = round(null_counts['null_sum']/df.shape[0], 3)
null_concerns = null_counts[null_counts['ratio']>thresh].sort_values('ratio', ascending=False)
null_concerns

#So, previous claims has quite a lot of nulls... and even occupation does. Interesting. I wonder if that's th same as unemployed.
#Credit score is always inteesting to see. Dependents are reasonable to ask, as well as feedback. Perhaps some similar issues iwth
#health score as well (ie on the surface I'd assume dubious data doesn't have a health score). No comment on income (in the
#context of nulls).

,null_sum,ratio
previous_claims,364029,0.30
occupation,358075,0.30
credit_score,137882,0.12
number_of_dependents,109672,0.09
customer_feedback,77824,0.07
health_score,74076,0.06
annual_income,44949,0.04


## Initial EDA

Note that this will be my first projec where I hopefully remember to get into the habit of pasting below into a bit of code to later preserve any changes made to the data to a cleaned csv.

In [31]:
i = -1

In [32]:
i += 1 #Eh...
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 0.


id
0         0.00
799988    0.00
800004    0.00
800003    0.00
800002    0.00
          ... 
399999    0.00
399998    0.00
399997    0.00
399996    0.00
1199999   0.00
Name: proportion, Length: 1200000, dtype: float64

In [33]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.") #Potentially redundant, but eh - I like to already see which
#feature it is. More intuitive and user-friendly.
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)[:10]

Now looking at feature 1, age.
Reminder that age has 18705 nulls (0.016%) - deal with them noob.


age
53.00   0.02
61.00   0.02
39.00   0.02
64.00   0.02
57.00   0.02
43.00   0.02
62.00   0.02
46.00   0.02
33.00   0.02
47.00   0.02
44.00   0.02
34.00   0.02
58.00   0.02
31.00   0.02
56.00   0.02
32.00   0.02
54.00   0.02
38.00   0.02
36.00   0.02
35.00   0.02
22.00   0.02
59.00   0.02
37.00   0.02
51.00   0.02
55.00   0.02
49.00   0.02
45.00   0.02
20.00   0.02
21.00   0.02
50.00   0.02
40.00   0.02
48.00   0.02
26.00   0.02
24.00   0.02
30.00   0.02
29.00   0.02
19.00   0.02
42.00   0.02
60.00   0.02
52.00   0.02
18.00   0.02
28.00   0.02
63.00   0.02
25.00   0.02
27.00   0.02
41.00   0.02
23.00   0.02
Name: proportion, dtype: float64

In [35]:
df[df.columns[i]].fillna(df[df.columns[i]].median(), inplace=True)

In [36]:
#Thanks to ChatGPT re. this neat bit right here.

pd.options.display.float_format = '{:,.2f}'.format
df['age'].describe()
# pd.reset_option('display.float_format')

count   1,200,000.00
mean           41.14
std            13.43
min            18.00
25%            30.00
50%            41.00
75%            53.00
max            64.00
Name: age, dtype: float64

Yeah, quite a nice distribution here. Shame it cuts off at 64. Unsurprising that it only starts at 18.

Now, immediately I'd want to band as we assume people of similar ages have similar enough characteristics. Furthermore, from an actuarial/rate perspective we'd do that all the time (for that or whatever other reasns). It definitely makes things simpler. I'm pretty sure even after banding we'd still consider specific ages.

So, starting from 18-22, simplifying all that to the '20' band.... and we'd likely either make the 60 band, consisting of people (by default) 58-62 to be bigger and also have 63 and 64... or a smaller 'old' band of anybody beyond that. Initially I was inclined a bigger final one, but my current desire is to have a smaller older band at our conclusion: It's flexible to account for a theortical change of data (that go beyond 64), retains spirit of my old work where we'd have a senior category, and it's funny (I plan on calling it 'oldies' or 'dinosaurs').

Anyways, essentially normally distributed around 41. Decent standard deviation, but makes sense given stuff (vague).

In [37]:
age_band_names = []
for band in range(20,65, 5):
    age_band_names.append(band)
# print(names)

for feature in age_band_names:
    cond1 = feature - 2
    cond2 = feature + 2
    df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]

#ChatGPT recommended something like this, but eh...
# for band in age_band_names[:-1]:  # Handle all except 'dinosaurs'
#     lower, upper = map(int, band.split('-'))
#     df[band] = ((df['age'] >= lower - 2) & (df['age'] <= upper + 2)).astype(int)
    
df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)#I'll keep their version. And they were right - why bother
#adding it earlier then have to deal with exception stuff if it's really just this...

In [39]:
pd.reset_option('display.float_format')

i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

#Nice even split. approximately.

Now looking at feature 2, gender.


gender
Male      0.502143
Female    0.497858
Name: proportion, dtype: float64

In [40]:
df['gender'] = [0 if gender == 'Female' else 1 for gender in df['gender']] #Recall that only a definite female
#we defined as male, with that gender abosrbing any 'other' - which would include an unknown. Especially in this data
#when it'd be the majority class.

In [42]:
pd.options.display.float_format = '{:,.2f}'.format

i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].describe()

Now looking at feature 3, annual_income.
Reminder that annual_income has 44949 nulls (0.037%) - deal with them noob.


count   1,155,051.00
mean       32,745.22
std        32,179.51
min             1.00
25%         8,001.00
50%        23,911.00
75%        44,634.00
max       149,997.00
Name: annual_income, dtype: float64

In [45]:
#Yeah, I should really keep the filling a seperate cell. Regardless. Let's analyze:
#Unexpectedly skewed to the right, with the median quite lower than the mean... Funny what year the date is based off of
#given inflation and the like. Regardless, the trend holds true.

#I'd expect a strong correlation to health, as by default if they can't even afford healthcare... At the same time, I recall
#the quandry of a certain streamer - if you're broke you get it for free and if you're rich it doesn't matter, but all the
#middl class.... And, evn thte upper tiers I could imagine depending upon their scenarios.... Ie even 150k is not all hat
#much in 2025....

df['annual_income'].fillna(df['annual_income'].median(), inplace=True) #Especially when either way I'd want to copy
#the code later on - I might as well get into the habit of using the precise name - as opposed to the variable is. Lest
#I mess up, want to change something, etc..

In [46]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 4, marital_status.
Reminder that marital_status has 18529 nulls (0.015%) - deal with them noob.


marital_status
Single     0.33
Married    0.33
Divorced   0.33
Name: proportion, dtype: float64

In [47]:
#Even enough distribution. Shame with the Pandas rounding as I'm sure it's not precise. Hmm, maybe I should just do both?
#So, in this case I'd assume single as definitionally they have themselves, so logically it would be the 'baseline'.

df['marital_status'].fillna('Single', inplace=True)
df = pd.get_dummies(df, columns=['marital_status'], dtype=int, drop_first=True)

In [48]:
#Guess I'll be lazy and not delete this line whenever I get rid of the i += 1 per a dummifying.
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 4, number_of_dependents.
Reminder that number_of_dependents has 109672 nulls (0.091%) - deal with them noob.


number_of_dependents
3.00   0.20
4.00   0.20
0.00   0.20
2.00   0.20
1.00   0.20
Name: proportion, dtype: float64

In [49]:
#We'll assume 0. Albeit not the median (2) I'd rather not add something when by default I can't prove its existence.
#I don't like this wording, but I'm trying to say: Unlike age where they must be something definitionally, there doesn't
#have to be dependents. Hence, without proof or any other additional considerations, I'd assume the default of 0.

df['number_of_dependents'].fillna(0, inplace=True)

In [50]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)
#Likely because it's synthetic, but eh - I'm not liking these even splits.... Phds should be much smaller... Oh well.

Now looking at feature 5, education_level.


education_level
Master's      0.25
PhD           0.25
Bachelor's    0.25
High School   0.24
Name: proportion, dtype: float64

In [51]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 6, occupation.
Reminder that occupation has 358075 nulls (0.298%) - deal with them noob.


occupation
Employed        0.34
Self-Employed   0.34
Unemployed      0.33
Name: proportion, dtype: float64

In [52]:
#Like number of dependents, I'll assume no job.
df['occupation'].fillna('Unemployed', inplace=True)
df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)

In [73]:
# i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")

#Tangential, but a bit tempted to try something like this:
#I'm getting tired of having to rechange the code dep. on the type of feature.
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 6, health_score.
Reminder that health_score has 74076 nulls (0.062%) - deal with them noob.
~~~
count   1,125,924.00
mean           25.61
std            12.20
min             2.01
25%            15.92
50%            24.58
75%            34.53
max            58.98
Name: health_score, dtype: float64


Tangential (again) but I wonder if for EDA such as this it might be better to just make an initia split of (suspected) categorical and numeric variables and go down with them. Eh... we'll see.

In [74]:
df['health_score'].fillna(df['health_score'].median(), inplace=True)

In [75]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
#Seriously, these splits are waaaay too even. One thing if this would be classification and we're talking about our targt,
#but for the features.... and every single one? I supposeit is ideal, but I don't want that in my data!!!

#Until data gets interesting I won't bother to comment. Ie even if realistically the groups should be skewed in whatever way,
#like expecting a minority of Phds...

Now looking at feature 7, location.
~~~
location
Suburban   0.33
Rural      0.33
Urban      0.33
Name: proportion, dtype: float64


In [76]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 8, policy_type.
~~~
policy_type
Premium         0.33
Comprehensive   0.33
Basic           0.33
Name: proportion, dtype: float64


In [77]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 9, previous_claims.
Reminder that previous_claims has 364029 nulls (0.303%) - deal with them noob.
~~~
previous_claims
0.00   0.37
1.00   0.36
2.00   0.20
3.00   0.06
4.00   0.01
5.00   0.00
6.00   0.00
7.00   0.00
8.00   0.00
9.00   0.00
Name: proportion, dtype: float64


In [86]:
#Makes sense to see such a skew, as unlikely for people to have more and more.
#Note that the max is actually 9 (well, we know that from our code logic already):
print(df['previous_claims'].describe())
atleast_1 = sum(df['previous_claims']>0)
print(df.shape[0] - atleast_1, atleast_1)

#Like before with dependents. See our logic there.
df['previous_claims'].fillna(0, inplace=True)

count   1,200,000.00
mean            0.70
std             0.94
min             0.00
25%             0.00
50%             0.00
75%             1.00
max             9.00
Name: previous_claims, dtype: float64
669462 530538


Wow what a mean considering all the 0s. Guess it's not that surprising as the 0s and 1s average out to be .5 then consider all the rest... Regardless, let's see (and comment in this block) if it's at least one:

Yeah, then this is quite interesting. Now we see a a hefty skew to the right (before the opposite, albeit not as exagerated). Ie you got those 'unhealthy' people always getting sick. Potentially they didn't get enough care. Regardless, the regulars dominate the activity here.

In [87]:
df[df['previous_claims']>0]['previous_claims'].describe()

count   530,538.00
mean          1.58
std           0.78
min           1.00
25%           1.00
50%           1.00
75%           2.00
max           9.00
Name: previous_claims, dtype: float64

In [88]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 10, vehicle_age.
Reminder that vehicle_age has 6 nulls (0.0%) - deal with them noob.
~~~
count   1,199,994.00
mean            9.57
std             5.78
min             0.00
25%             5.00
50%            10.00
75%            15.00
max            19.00
Name: vehicle_age, dtype: float64


In [93]:
#Bit to the lft, but overall nice distribution. Honestly should be expected with this dataset by now...

#Of note are these babies - notice how we have only whole numbers so likly they're just very new cars.

sum(df['vehicle_age']==0), round(sum(df['vehicle_age']==0)/df.shape[0], 3)

(61615, 0.051)

In [89]:
df['vehicle_age'].fillna(df['vehicle_age'].median(), inplace=True) #Like human age must be something. In this case with so
#Few nulls though a bit negligible.

0         17.00
1         12.00
2         14.00
3          0.00
4          8.00
           ... 
1199995    5.00
1199996   10.00
1199997   19.00
1199998    7.00
1199999   18.00
Name: vehicle_age, Length: 1200000, dtype: float64

In [94]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Again, another clean even (ess.) dist.. Recall that the max is 300 and absolute is 850..

Now looking at feature 11, credit_score.
Reminder that credit_score has 137882 nulls (0.115%) - deal with them noob.
~~~
count   1,062,118.00
mean          592.92
std           149.98
min           300.00
25%           468.00
50%           595.00
75%           721.00
max           849.00
Name: credit_score, dtype: float64


In [ ]:
df['credit_score'].fillna(df['credit_score'].median(), inplace=True)

In [98]:
#Just for kicks, I want to see how evenly distributed some of the variables are:
df[df['credit_score']<350]['education_level'].value_counts(normalize=True)

education_level
Master's      0.26
PhD           0.25
Bachelor's    0.25
High School   0.24
Name: proportion, dtype: float64

In [99]:
df[df['credit_score']>800]['education_level'].value_counts(normalize=True)
#Unsurprising....

education_level
PhD           0.26
Bachelor's    0.25
Master's      0.25
High School   0.24
Name: proportion, dtype: float64

In [ ]:
Oh gf me - forgot to dummify this stuff. Given this last bit of code it might be better to dummify at the end.
Or, do something like keeping the original. Do my EDA, then redo the dummification/actually do it at the end.
Especially when either way we'd need to do so for the test data we might as well not get ourselves confused with that (
recall the +1 potential disasters of our i-scrolling) & avoid this issue [easily compare all of a feature's categories
together].

From ChatGPT I'll sort out later.... Enough of the coding and analysis practice for today.

df['occupation_original'] = df['occupation']  # Preserve original column
df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)

In [ ]:
original_col = df['occupation']
df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)
df.insert(0, 'occupation_original', original_col)  # Insert at the start

In [ ]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

## Concluding Matters

In [ ]:
stop

In [ ]:
df['annual_income'].fillna(df['annual_income'].median(), inplace=True)

In [ ]:
#As I said earlier, prepping the total changed cleanings for modeling.
#Ie kind of like on Git we have the commit before actually pushing.
df = pd.read_csv('../data/train_data.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')
for col in df.columns:
    try:
        df[col] = round(df[col], 3)
    except:
        pass

df['age'].fillna(df['age'].median(), inplace=True)

age_band_names = []
for band in range(20,65, 5):
    age_band_names.append(band)
for feature in age_band_names:
    cond1 = feature - 2
    cond2 = feature + 2
    df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]
df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)

df['gender'] = [0 if gender == 'Female' else 1 for gender in df['gender']]

df['annual_income'].fillna(df['annual_income'].median(), inplace=True)

df['marital_status'].fillna('Single', inplace=True)
df = pd.get_dummies(df, columns=['marital_status'], dtype=int, drop_first=True)

df['number_of_dependents'].fillna(0, inplace=True)

df['occupation'].fillna('Unemployed', inplace=True)
df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)

df['health_score'].fillna(df['health_score'].median(), inplace=True)

df['previous_claims'].fillna(0, inplace=True)

df['vehicle_age'].fillna(df['vehicle_age'].median(), inplace=True)

df['credit_score'].fillna(df['credit_score'].median(), inplace=True)